In [1]:
import numpy as np
import re

def check_scanner_pair(scanner_a, scanner_b, data):
    for ref_a in range(data[scanner_a].shape[0]):
        distance_difference_a = data[scanner_a] - data[scanner_a][ref_a,:]
        distance_difference_a = np.delete(distance_difference_a, ref_a, axis=0)

        for ref_b in range(data[scanner_b].shape[0]):
            distance_difference_b = data[scanner_b] - data[scanner_b][ref_b,:]
            distance_difference_b = np.delete(distance_difference_b, ref_b, axis=0)  # remove reference line

            axis_assignment = [None, None, None]
            axis_sign = [None, None, None]
            for axis_a in range(3):
                for axis_b in range(3):
                    d1 = set(distance_difference_a[:, axis_a])
                    d2 = set(distance_difference_b[:, axis_b])
                    d3 = set(-1 * distance_difference_b[:, axis_b])
                    common_distances = set.intersection(d1, d2)
                    if len(common_distances) > MIN_SATS:
                        axis_assignment[axis_a] = axis_b
                        axis_sign[axis_a] = 1
                    common_distances = set.intersection(d1, d3)
                    if len(common_distances) > MIN_SATS:
                        axis_assignment[axis_a] = axis_b
                        axis_sign[axis_a] = -1
            if not any(item is None for item in axis_assignment):
                return {
                    'scanner_b': scanner_b,
                    'ref_a': ref_a,
                    'ref_b': ref_b,
                    'axis_assignment': axis_assignment,
                    'axis_sign': axis_sign,
                }
    return None

def apply_result(result, data):
    scanner_b = result["scanner_b"]
    ref_a = result["ref_a"]
    ref_b = result["ref_b"]
    axis_assignment = result["axis_assignment"]
    axis_sign = result["axis_sign"]

    # rotate new beacons
    vec_1_to_newbeacons_1 = data[scanner_b]
    rows = [(axis_sign[i] * vec_1_to_newbeacons_1[:, axis_assignment[i]]).reshape((-1, 1)) for i in range(3)]
    vec_1_to_newbeacons_0 = np.concatenate(rows, axis=1)  # n x 3

    # find scanner location
    vec_0_to_refA_0 = data[0][ref_a, :]
    vec_1_to_refB_0 = vec_1_to_newbeacons_0[ref_b, :]
    vec_0_to_1_0 = vec_0_to_refA_0 - vec_1_to_refB_0
    all_scanners_0.append(vec_0_to_1_0)

    # calculate beacon location as seen from scanner 0
    vec_0_to_newbeacons_0 = vec_0_to_1_0 + vec_1_to_newbeacons_0
    for line in vec_0_to_newbeacons_0:
        all_beacons_0.add(tuple(line))

    # base next search on just the newly added points
    return vec_0_to_newbeacons_0

with open('input19') as file:
    lines = file.read().splitlines()

MIN_SATS = 10

n_scanners = 0
data = []
for line in lines:
    if "scanner" in line:
        data.append([])
        scanner_id = int(re.search("\d+", line)[0])
        if scanner_id+1 > n_scanners:
            n_scanners = scanner_id+1
    elif len(line) == 0:
        pass
    else:
        numbers = line.split(',')
        xyz = tuple(map(int, numbers))
        xyz_array = np.array(xyz, dtype=int)
        data[scanner_id].append(xyz_array)
    
for i in range(len(data)):
    data[i] = np.concatenate(data[i]).reshape((-1, 3))

# ─── Apply Satelite 0 ─────────────────────────────────────────────────────────
all_scanners_0 = [(0, 0, 0)]
all_beacons_0 = set(tuple(line) for line in data[0])

# ─── Apply All Satelites ──────────────────────────────────────────────────────
scanners_remaining = set(range(1, n_scanners))
while len(scanners_remaining) > 0:
    found = None
    for scanner_idx in scanners_remaining:
        result = check_scanner_pair(0, scanner_idx, data)
        if result is not None:
            found = True
            scanners_remaining.remove(scanner_idx)
            data[0] = apply_result(result, data)
            break
    if not found:
        data[0] = np.array(tuple(item for item in all_beacons_0))

print('result 1: ', len(all_beacons_0))

max_distance = 0
for i in range(len(all_scanners_0)):
    for j in range(i):
        if i == j:
            continue
        distance = np.sum(np.abs(all_scanners_0[i]-all_scanners_0[j]))
        if distance > max_distance:
            max_distance = distance
print('result 2: ', max_distance)

result 1:  342
result 2:  9668
